## 用 Keras 建構模型來預測 Titanic 資料集，並將結果與 HW03 比較

### 1.資料缺失值處理與特徵選取(同HW3)

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [99]:
# 載入資料
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

#### 觀察資料

In [100]:
train.head() 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [101]:
train.info() # Age、Cabin、Embarked有缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [102]:
test.info() # Age、Fare、Cabin有缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [103]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [104]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


#### 選特徵&補缺值

In [105]:
Y_train = train['Survived']
X_train = train.append(test) #合併表格處理，之後會再分開
X_train.shape

(1309, 12)

In [106]:
X_train['Family'] = X_train['SibSp'] + X_train['Parch'] # 把親屬合併起來，成為一個新特徵

X_train.drop('SibSp', 1, inplace=True)
X_train.drop('Parch', 1, inplace=True)

In [107]:
# 選特徵(我覺得簡單重要的&網站已分析的)
selected_features = ['Pclass', 'Age', 'Family', 'Fare', 'Sex', 'Embarked']

# 補缺值(簡單做或參考網站作法)，之後應仔細分析資料再個別處理
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_train['Fare'].fillna(X_train['Fare'].median(), inplace=True)
X_train['Embarked'].fillna("C", inplace=True)

In [108]:
# One-Hot Encoding
X_train['Pclass'] = X_train['Pclass'].astype('category').cat.codes
X_train['Sex'] = X_train['Sex'].astype('category').cat.codes
X_train['Embarked'] = X_train['Embarked'].astype('category').cat.codes

In [109]:
# 準備訓練
X_test = X_train.iloc[891:]
X_train = X_train.head(891)

X_train.shape, Y_train.shape, X_test.shape

((891, 11), (891,), (418, 11))

In [110]:
X_train = X_train[selected_features]
X_test = X_test[selected_features]
X_train.head()

,Pclass,Age,Family,Fare,Sex,Embarked
0,2,22.0,1,7.2500,1,2
1,0,38.0,1,71.2833,0,0
2,2,26.0,0,7.9250,0,2
3,0,35.0,1,53.1000,0,2
4,2,35.0,0,8.0500,1,2


### 2.DecisionTree(同HW3)

In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

tree_clf = DecisionTreeClassifier(max_depth=3)
tree_clf.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [112]:
predict = tree_clf.predict(X_test)

#### 成績
![img](https://i.imgur.com/D8Ue5wD.jpg)

### 3.Keras
#### [參考資料1.將TensorFlow與Keras安裝在Windows系統](http://tensorflowkeras.blogspot.com/2017/08/tensorflowkeraswindows_29.html)
#### [參考資料2.Keras深度學習(Deep Learning) 預測鐵達尼號旅客生存機率](http://tensorflowkeras.blogspot.com/2017/09/kerasdeep-learning_32.html)

In [113]:
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.layers import Dense, Activation

In [114]:
# 建立模型
model = Sequential()

In [115]:
# 第一層
model.add(Dense(units = 20, input_dim = 6, kernel_initializer = 'uniform', activation='relu'))
# 第二層
model.add(Dense(units = 30, kernel_initializer = 'uniform', activation='relu'))
# 輸出層
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [116]:
# 訓練模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [123]:
model.fit(X_train, Y_train, validation_split=0.2, epochs=50)

Train on 712 samples, validate on 179 samples
Epoch 1/50
712/712 [==============================] - 0s 95us/step - loss: 0.4524 - acc: 0.8090 - val_loss: 0.3512 - val_acc: 0.8547
Epoch 2/50
712/712 [==============================] - 0s 112us/step - loss: 0.4338 - acc: 0.8034 - val_loss: 0.3547 - val_acc: 0.8492
Epoch 3/50
712/712 [==============================] - 0s 86us/step - loss: 0.4376 - acc: 0.8006 - val_loss: 0.3517 - val_acc: 0.8436
Epoch 4/50
712/712 [==============================] - 0s 79us/step - loss: 0.4364 - acc: 0.8090 - val_loss: 0.3559 - val_acc: 0.8380
Epoch 5/50
712/712 [==============================] - 0s 95us/step - loss: 0.4399 - acc: 0.8034 - val_loss: 0.3483 - val_acc: 0.8492
Epoch 6/50
712/712 [==============================] - 0s 97us/step - loss: 0.4356 - acc: 0.8090 - val_loss: 0.3565 - val_acc: 0.8492
Epoch 7/50
712/712 [==============================] - 0s 111us/step - loss: 0.4381 - acc: 0.8090 - val_loss: 0.3464 - val_acc: 0.8436
Epoch 8/50
712/712 [=

In [124]:
# 測試模型
acc = model.evaluate(X_train, Y_train)
acc[1]

891/891 [==============================] - 0s 58us/step


0.8226711562051784

In [125]:
# 預測模型
survived_predict = model.predict(X_test)
survived_predict = np.where(survived_predict > 0.5 , 1 ,0)
survived_predict = survived_predict.flatten()# 轉成1維
survived_predict

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [126]:
# 將結果輸出成規定的格式
submission = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict})
submission.to_csv('submission.csv', index=False)

#### 成績 >> 相較HW3決策樹結果沒有改善
![img](https://i.imgur.com/7OKadBp.jpg)